In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import cProfile
from collections import deque, Counter
from time import time

path = '/home/mariano/Descargas/VID_20170720_132629833.mp4'

from calculo.video import centroid  # Cython

def center_of_square(contour):
    return centroid(*cv2.boundingRect(contour))

def roi_dsg(array, widths=(1.2, 1.2, 1.4)):
    y_cord = array[:, 1]
    top = min(y_cord)
    bottom = max(y_cord)
    distance = (bottom - top) / 6.0
    regions = []
    for seg, width in zip((1, 3, 6), widths):
        middle = (top + distance * seg)
        band_width = distance * width
        regions.append((middle - band_width, middle + band_width))
    return regions

def set_rois(markers, **kwargs):
    roi = False
    n = 0
    while not roi:
        marker = markers[n]
        if len(marker) is 7:
            roi = roi_dsg(marker, **kwargs)
        n += 1
    return roi

def partitionate(markers, regions_of_insterest):
        Y = markers[:, 1] if markers.any() else ()
        regions = ('trunk', 'knee', 'ankle')
        partitions = {}
        # Separo los marcadores por regiones.
        for roi, n, region in zip(regions_of_insterest, (2, 2, 3), regions):
            marker = markers[np.logical_and(Y > roi[0], Y < roi[1])]
            if marker.shape[0] != n:
                marker = None
            partitions[region] = marker
        return partitions

In [2]:
markers = deque(maxlen=2500)  # Límite de data que se permite almacenar.
t1 = time()
vid = cv2.VideoCapture(path)
is_frame, frame = vid.read()
c = 0
while is_frame:
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    binary = cv2.threshold(gray, 240., 255., cv2.THRESH_BINARY)[1]
    blur = cv2.GaussianBlur(binary, (5, 5), 0)
    contours= cv2.findContours(blur, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[1]
    markers.append(np.array(map(center_of_square, contours)))
    is_frame, frame = vid.read()

print time() - t1

25.8548719883


In [3]:
t1 = time()
rois = set_rois(markers)
for __ in xrange(len(markers)):
    markers.append(partitionate(markers.popleft(), rois))
print time() - t1

0.0652470588684


In [4]:
#Necesito corregir los marcadores de tobillo.
ankle = [partition['ankle'] for partition in markers]

def diff(array):
    prev = array[0]
    vel = []
    for new in array:
        if new is None:
            vel.append(None)
        else:
            vel.append(new - prev)
            prev = new
    return vel

p1 = map(lambda x: x[0, 1] if isinstance(x, np.ndarray) else None, ankle)
# p2 = map(lambda x: x[1, 1] if isinstance(x, np.ndarray) else None, ankle)

vel = diff(p1)

In [60]:
def foot_markers_order(ankle_array):
    if ankle_array is None:
        return ankle_array
    else:
        x1, x2, refer = ankle_array[:, 0]
        values = ((abs(refer - x1), 0),
                  (abs(refer - x2), 1))
        talon = ankle_array[min(values)[1]]
        antepie = ankle_array[max(values)[1]]
        maleolo = ankle_array[-1]
    return np.array((maleolo, talon, antepie))


def get_direction(ankle_array):
    x_talon = ankle_array[1, 0]
    x_antepie = ankle_array[2, 0]
    if x_talon < x_antepie:
        return 1
    else:
        return -1

In [213]:
def set_direction(ankle_velocities, none_limit=25):
    values = []
    buff = []
    for i, vel in enumerate(ankle_velocities):
        if vel == 0:
            # establecer la dirección. Agregar a la lista de salida.
            ordered = foot_markers_order(ankle[i])
            direction = get_direction(ordered)
            values.append((i, direction))
        elif vel is None:
            # agregar a la lista de salida
            values.append((i, None))
        else:
            #guardar en buffer
            buff.append((i, 0))
    directions = []
    for i, val in values + buff:
        directions.insert(i, val)
    # En la lista directions existen valores de direcición, de velocidad != 0
    # y None. Para una lista homogénea de direcciones se vuelve a procesar.
    none_values = 0
    rare_indices = []
    dire_indices = []
    dire_values = []
    for i, val in enumerate(directions):
        if val is None:
            if none_values < none_limit:
                # Se puede estar dentro de la imagen que tiene marcadores
                rare_indices.append(i)
                none_values += 1
            else:
                # La imagen ya no tiene marcadores (según el límite de
                # valores None que establece el usuario*; Si hay valores de
                # dirección se reemplazan en el arreglo original.
                if dire_values != []:
                    # Como existe mas de un valor que del otro en direcciones
                    # (ver mas abajo), el signo de la suma establece la
                    # dirección correcta (y final) de los intervalos del
                    # arreglo.
                    freq = 1 if sum(dire_values) > 0 else -1
                    for i  in dire_indices + rare_indices[:-none_limit]:
                        directions.pop(i)
                        directions.insert(i, freq)
                    dire_values = []
                    dire_indices = []
                    rare_indices = []
        elif val == 0:
            # Entonces la dirección es desconocida y estos valores se
            # tienen que reemplazar por el valor de dirección.
            rare_indices.append(i)
            none_values = 0
        else:
            # Si no es None, y tampoco es 0, entonces es un valor de
            # dirección (1 ó -1). Pueden coexistir ambos, porque en
            # la primer lectura de direcciones hay intercambio de
            # marcadores de talon y antepie. Sin embargo como la mayor
            # parte de las direcciones se toman en apoyo, simpre va
            # a haber mas de un valor que del otro.
            dire_values.append(val)
            dire_indices.append(i)
            none_values = 0
    return directions

In [214]:
for i in zip(ankle, set_direction(vel)):
    print i

(array([[ 227, 1045],
       [ 157, 1041],
       [ 170,  988]]), 1)
(array([[ 227, 1045],
       [ 157, 1041],
       [ 170,  988]]), 1)
(array([[ 227, 1045],
       [ 157, 1041],
       [ 172,  988]]), 1)
(array([[ 228, 1045],
       [ 158, 1040],
       [ 174,  988]]), 1)
(array([[ 228, 1045],
       [ 158, 1040],
       [ 176,  988]]), 1)
(array([[ 228, 1045],
       [ 158, 1040],
       [ 178,  988]]), 1)
(array([[ 229, 1045],
       [ 159, 1039],
       [ 180,  987]]), 1)
(array([[ 230, 1045],
       [ 160, 1037],
       [ 184,  985]]), 1)
(array([[ 231, 1045],
       [ 163, 1033],
       [ 189,  980]]), 1)
(array([[ 234, 1044],
       [ 167, 1027],
       [ 194,  974]]), 1)
(array([[ 239, 1044],
       [ 173, 1018],
       [ 205,  965]]), 1)
(array([[ 246, 1041],
       [ 185, 1005],
       [ 222,  952]]), 1)
(array([[ 258, 1035],
       [ 205,  985],
       [ 250,  938]]), 1)
(array([[ 278, 1023],
       [ 238,  962],
       [ 288,  921]]), 1)
(array([[ 314, 1007],
       [ 280

from time import sleep

par_vel_mark = zip(vel, ankle)
vid = cv2.VideoCapture(path)
n = 0
while True:
    is_frame, frame = vid.read()
    if is_frame is False:
        break
    if n in l:
        __, mark = par_vel_mark[n]
        for m in mark:
            cv2.circle(frame, (m[0], m[1]), 7, (0, 0, 255), -1)
    cv2.imshow(path, cv2.resize(frame, None, fx=.4, fy=.4))
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    n += 1
    sleep(.15)
vid.release()
cv2.destroyAllWindows()